Hard задачи с Stratascretch по SQL

63 задача:Find the top 5 states with the most 5 star businesses. Output the state name along with the number of 5-star businesses and order records by the number of 5-star businesses in descending order. In case there are ties in the number of businesses, return all the unique states. If two states have the same result, sort them in alphabetical order.



In [ ]:
with aka as (select state, count(business_id) as n_business from yelp_business
where stars=5
group by state
order by 2 desc, 1 asc)

select state, n_business
from
(select state, n_business, rank() over(order by n_business desc, state asc) as ranged
from aka) as koko
where ranged <=5

64 задача :Find the popularity percentage for each user on Meta/Facebook. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform, then converted into a percentage by multiplying by 100.
Output each user along with their popularity percentage. Order records in ascending order by user id.
The 'user1' and 'user2' column are pairs of friends.

In [ ]:
with aka as (select user1, count(user2) as counti from facebook_friends
group by user1),
oso as (select user2, count(user1) as counti from facebook_friends
group by user2),
haha as (
select user1, counti
from aka
union
select user2, counti
from oso)

select user1,(sum(counti)::decimal/count(*) over())*100 as popularity_percent
from haha
group by user1
order by 2 desc

65 задание:Find the number of times the words 'bull' and 'bear' occur in the contents. We're counting the number of times the words occur so words like 'bullish' should not be included in our count.
Output the word 'bull' and 'bear' along with the corresponding number of occurrences.

In [ ]:
SELECT
    word,nentry
FROM
    ts_stat('SELECT to_tsvector(contents) FROM google_file_store')
where word ilike 'bull' or word ilike 'bear'

66 задание:ABC Corp is a mid-sized insurer in the US and in the recent past their fraudulent claims have increased significantly for their personal auto insurance portfolio. They have developed a ML based predictive model to identify propensity of fraudulent claims. Now, they assign highly experienced claim adjusters for top 5 percentile of claims identified by the model.
Your objective is to identify the top 5 percentile of claims from each state. Your output should be policy number, state, claim cost, and fraud score.

In [ ]:
with aka as (select state, percentile_cont(0.05) within group (order by fraud_score desc) as percentile
from fraud_score
group by state)
select policy_num, f.state, claim_cost, fraud_score
from fraud_score f join aka p on f.state=p.state
where fraud_score >= percentile

67 задание:Select the most popular client_id based on a count of the
number of users who have at least 50% of their events from the following list: 'video call received', 'video call sent', 'voice call received', 'voice call sent'.



In [ ]:
with oko as (select user_id, proc
from (select user_id,count(event_id) filter( where event_type in ('video call received', 'video call sent', 'voice call received', 'voice call sent'))/count(*)::decimal)*100 as proc
from fact_events
group by user_id) as koko
where proc>=50

select client_id
from (select client_id, count(*)
from fact_events inner join oko using ("user_id")
group by client_id
order by 2 desc) as kok
limit 1

68 задание:You're given a dataset of searches for properties on Airbnb. For simplicity, let's say that each search result (i.e., each row) represents a unique host. Find the city with the most amenities across all their host's properties. Output the name of the city.



In [ ]:
with aka as (select city, sum(array_length(string_to_array(amenities, ','),1)) as check
from airbnb_search_details
group by city
order by 2 desc
limit 1)
select city
from aka

69 здание:Given a table of purchases by date, calculate the month-over-month percentage change in revenue. The output should include the year-month date (YYYY-MM) and percentage change, rounded to the 2nd decimal point, and sorted from the beginning of the year to the end of the year.
The percentage change column will be populated from the 2nd month forward and can be calculated as ((this month's revenue - last month's revenue) / last month's revenue)*100.

In [ ]:
select to_char(created_at::date,'YYYY-MM') as year_month, round((sum(value)-lag(sum(value)) over(order by to_char(created_at::date,'YYYY-MM')))/lag(sum(value)) over(order by to_char(created_at::date,'YYYY-MM'))*100,2) as revenue_diff_pct
from sf_transactions
group by to_char(created_at::date,'YYYY-MM')
order by 1 asc

70 задание:You’re given a table of rental property searches by users. The table consists of search results and outputs host information for searchers. Find the minimum, average, maximum rental prices for each host’s popularity rating. The host’s popularity rating is defined as below:
0 reviews: New
1 to 5 reviews: Rising
6 to 15 reviews: Trending Up
16 to 40 reviews: Popular
more than 40 reviews: Hot


Tip: The id column in the table refers to the search ID. You'll need to create your own host_id by concating price, room_type, host_since, zipcode, and number_of_reviews.


Output host popularity rating and their minimum, average and maximum rental prices.

In [ ]:
WITH hosts AS
  (SELECT DISTINCT concat(price, room_type, host_since, zipcode, number_of_reviews) AS host_id,
          number_of_reviews,
          price
   FROM airbnb_host_searches)
SELECT host_popularity AS host_pop_rating,
       min(price) AS min_price,
       avg(price) AS avg_price,
       max(price) AS max_price
FROM
  (SELECT CASE
              WHEN number_of_reviews = 0 THEN 'New'
              WHEN number_of_reviews BETWEEN 1 AND 5 THEN 'Rising'
              WHEN number_of_reviews BETWEEN 6 AND 15 THEN 'Trending Up'
              WHEN number_of_reviews BETWEEN 16 AND 40 THEN 'Popular'
              WHEN number_of_reviews > 40 THEN 'Hot'
          END AS host_popularity,
          price
   FROM hosts) a
GROUP BY host_pop_rating

71 задание:Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.



In [ ]:
with aka as (select date, sum(downloads) filter( where paying_customer='no') as non_paying,
sum(downloads) filter( where paying_customer='yes') as paying
from ms_user_dimension inner join ms_download_facts using (user_id)
inner join ms_acc_dimension using (acc_id)
group by date)
select date, non_paying, paying
from aka
where non_paying>paying
order by date asc

72 задание:You have a table of in-app purchases by user. Users that make their first in-app purchase are placed in a marketing campaign where they see call-to-actions for more in-app purchases. Find the number of users that made additional in-app purchases due to the success of the marketing campaign.

The marketing campaign doesn't start until one day after the initial in-app purchase so users that only made one or multiple purchases on the first day do not count, nor do we count users that over time purchase only the products they purchased on the first day.

In [ ]:
select *
from
(SELECT  *,
            DENSE_RANK() OVER (PARTITION BY user_id ORDER BY created_at) AS rnk_1,
            DENSE_RANK() OVER (PARTITION BY user_id, product_id ORDER BY created_at)
            AS rnk_2
    FROM marketing_campaign) as koko